In [1]:
from sklearn.linear_model import PoissonRegressor
import numpy as np
import pandas as pd
from scipy.special import factorial

In [5]:
df = pd.read_csv('data.csv')
X = df.age.values.reshape(-1,1)
y = df.accidents.values
y_skl = y/df.personyears.values   # need to handle exposure manually in skl

In [6]:
df.head()

name  personyears  age  accidents
0    A            6   34          4
1    B            5   38          3
2    C            4   45          0
3    D            3   25         15
4    E            2   21         18

In [7]:
res = PoissonRegressor(alpha=0.0).fit(X,y_skl)
beta = np.array([res.intercept_, res.coef_[0]]).reshape(-1,1)
print('beta = ', beta.T)
print('fitted occurance rate = ', res.predict(X))
print('fitted y = ', df.personyears.values*res.predict(X))

beta =  [[ 6.10267484 -0.1839723 ]]
fitted occurance rate =  [0.85865918 0.41136582 0.1134857  4.49680641 9.38635125]
fitted y =  [ 5.15195507  2.05682911  0.4539428  13.49041923 18.77270251]


In [8]:
mu = (df.personyears.values * np.exp(beta[0] + X.dot(beta[1]))).reshape(-1,1)
log_mu = np.log(mu)
l = y.reshape(-1,1)*log_mu - mu - np.log(factorial(y.reshape(-1,1)))
log_likelihood = l.sum()
print('log_likelihood = ', log_likelihood)#match

log_likelihood =  -8.656411974760776


In [9]:
# matches R output
import statsmodels.api as sm
X_c = np.hstack((np.ones((X.shape[0],1)), X))
sm_res=sm.GLM(y, X_c, family = sm.families.Poisson(), link='log', exposure = df.personyears.values).fit()
print('beta = ', sm_res.params)
print('log-likelihood = ', sm.GLM(y, X_c,family = sm.families.Poisson(), link='log', exposure = df.personyears.values).loglike(sm_res.params))
print('fitted y = ', df.personyears.values*sm_res.predict(X_c))

beta = sm_res.params.reshape(-1,1)
mu = (df.personyears.values * np.exp(beta[0] + X.dot(beta[1]))).reshape(-1,1)
log_mu = np.log(mu)
l = y.reshape(-1,1)*log_mu - mu - np.log(factorial(y.reshape(-1,1)))
log_likelihood = l.sum()
print('log_likelihood = ', log_likelihood)#match


beta =  [ 6.18835659 -0.18730852]
log-likelihood =  -8.649522123148902
fitted y =  [ 5.01095509  1.97401771  0.42560983 13.5211617  19.06825567]
log_likelihood =  -8.649522123148902


C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['link']
  warnings.warn(msg, ValueWarning)
C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['link']
  warnings.warn(msg, ValueWarning)


In [10]:
print(sm_res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                    5
Model:                            GLM   Df Residuals:                        3
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -8.6495
Date:                Mon, 09 May 2022   Deviance:                       1.7469
Time:                        16:02:32   Pearson chi2:                     1.38
No. Iterations:                     5   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.1884      0.737      8.401      0.0

C:\Users\Woon\Documents\env_ds_3_10\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['link']
  warnings.warn(msg, ValueWarning)


In [20]:
sm_res.pseudo_rsquared()

C:\Users\Zoom\Documents\env_ds_310\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['link']
  warnings.warn(msg, ValueWarning)


0.9999971698310279